# 04. SQL 기반 분석

> 본 노트북은 Docker 환경에서 구동된 MySQL 서버(`abtest` DB)에 접근하여
> SQL 쿼리로 데이터를 집계하고 Tableau로 연계하여 시각화를 수행한다.
>
> 이전 단계 (`01_data_overview`, `02_eda`, `03_ab_test`)에서 과정을 SQL 기반으로 재현하고 결과 테이블을 Tableau에서 시각적으로 해석한다.

### 0. 환경 설정 및 DB 연결

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, text

# 연결 정보
user = "root"
password = "1234"   # 컨테이너 만들 때 설정한 root 비밀번호
host = "localhost"
port = 3306
database = "abtest"

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# 연결 테스트
with engine.connect() as conn:
    result = conn.execute(text("SELECT DATABASE();"))  # text()로 감싸야 함
    print("현재 DB:", result.fetchone()[0])

현재 DB: abtest


### 1. 데이터 개요
> 01_data_overview

In [14]:
query = """
SELECT * FROM marketing_ab LIMIT 5;
"""

# 데이터 확인
df_overview = pd.read_sql(query, engine)
df_overview

,user_id,test_group,converted,total_ads,most_ads_day,most_ads_hour
0,1069124,ad,0,130,Monday,20
1,1119715,ad,0,93,Tuesday,22
2,1144181,ad,0,21,Tuesday,18
3,1435133,ad,0,355,Tuesday,10
4,1015700,ad,0,276,Friday,14


In [35]:
# 결측치 확인
query = """
SELECT 
    SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) user_id,
    SUM(CASE WHEN test_group IS NULL THEN 1 ELSE 0 END) test_group,
    SUM(CASE WHEN converted IS NULL THEN 1 ELSE 0 END) converted,
    SUM(CASE WHEN most_ads_day IS NULL THEN 1 ELSE 0 END) most_ads_day,
    SUM(CASE WHEN most_ads_hour IS NULL THEN 1 ELSE 0 END) most_ads_hour,
    SUM(CASE WHEN total_ads IS NULL THEN 1 ELSE 0 END) total_ads
FROM marketing_ab;
"""

df_nulls = pd.read_sql(query, engine)
df_nulls

,user_id,test_group,converted,most_ads_day,most_ads_hour,total_ads
0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# 중복 데이터 확인
query = """
SELECT COUNT(user_id) - COUNT(DISTINCT user_id) user_id
FROM marketing_ab;
"""

df_overview = pd.read_sql(query, engine)
df_overview

,user_id
0,0


### 2. 탐색적 데이터 분석 (EDA)
> 02_eda

#### 2-1. 단변량 탐색
<hr>

In [45]:
# 그룹 분포
query = """
SELECT test_group, COUNT(*) AS n
FROM marketing_ab
GROUP BY test_group;
"""

df_group = pd.read_sql(query, engine)
display(df_group)

,test_group,n
0,ad,564577
1,psa,23524


In [49]:
# 전환 여부 분포
query = """
SELECT converted, COUNT(*) AS n
FROM marketing_ab
GROUP BY converted;
"""

df_conv = pd.read_sql(query, engine)
display(df_conv)

,converted,n
0,0,573258
1,1,14843


In [48]:
# 요일 분포
query = """
SELECT most_ads_day, COUNT(*) AS n
FROM marketing_ab
GROUP BY most_ads_day;
"""

df_day = pd.read_sql(query, engine)
display(df_day)

,most_ads_day,n
0,Monday,87073
1,Tuesday,77479
2,Friday,92608
3,Saturday,81660
4,Wednesday,80908
5,Sunday,85391
6,Thursday,82982


In [57]:
# 시간대 분포 Top 5
query = """
SELECT most_ads_hour, COUNT(*) AS n
FROM marketing_ab
GROUP BY most_ads_hour
ORDER BY n DESC;
"""

df_hour = pd.read_sql(query, engine)
display(df_hour)

,most_ads_hour,n
0,13,47655
1,12,47298
2,11,46210
3,14,45648
4,15,44683
5,10,38939
6,16,37567
7,17,34988
8,18,32323
9,9,31004


In [44]:
# 광고 노출량 분포
df_ads = pd.read_sql("""
SELECT MIN(total_ads) AS min_ads,
       MAX(total_ads) AS max_ads,
       AVG(total_ads) AS avg_ads
FROM marketing_ab;
""", engine)
display(df_ads)

,min_ads,max_ads,avg_ads
0,1,2065,24.8209


#### 2-2. 이변량 탐색
<hr>

In [59]:
# 그룹별 전환율
query = """
SELECT test_group,
       SUM(converted)*1.0/COUNT(*) AS conversion_rate,
       COUNT(*) AS n
FROM marketing_ab
GROUP BY test_group;
"""

df_conv_rate = pd.read_sql(query, engine)
display(df_conv_rate)

,test_group,conversion_rate,n
0,ad,0.02555,564577
1,psa,0.01785,23524


In [60]:
# 요일별 전환율
query = """
SELECT most_ads_day,
       SUM(converted)*1.0/COUNT(*) AS conversion_rate,
       COUNT(*) AS n
FROM marketing_ab
GROUP BY most_ads_day
ORDER BY most_ads_day;
"""

df_day_rate = pd.read_sql(query, engine)
display(df_day_rate)

,most_ads_day,conversion_rate,n
0,Friday,0.02221,92608
1,Monday,0.03281,87073
2,Saturday,0.02105,81660
3,Sunday,0.02448,85391
4,Thursday,0.02157,82982
5,Tuesday,0.02984,77479
6,Wednesday,0.02494,80908


In [61]:
# 시간대별 전환율
query = """
SELECT most_ads_hour,
       SUM(converted)*1.0/COUNT(*) AS conversion_rate,
       COUNT(*) AS n
FROM marketing_ab
GROUP BY most_ads_hour
ORDER BY most_ads_hour;
"""

df_hour_rate = pd.read_sql(query, engine)
display(df_hour_rate)

,most_ads_hour,conversion_rate,n
0,0,0.01842,5536
1,1,0.01291,4802
2,2,0.00731,5333
3,3,0.01045,2679
4,4,0.01524,722
5,5,0.02092,765
6,6,0.02224,2068
7,7,0.01811,6405
8,8,0.01952,17627
9,9,0.01919,31004


### 3. A/B Test
> 03_ab_test

#### 3-1. 전체 효과
- **H0 (귀무가설):** 광고 그룹과 PSA 그룹의 전환율은 차이가 없다.
- **H1 (대립가설):** 광고 그룹과 PSA 그룹의 전환율은 차이가 있다.
<hr>

In [71]:
# 교차표 (그룹 × 전환)
df_ab = pd.read_sql("""
SELECT test_group, converted, COUNT(*) AS cnt
FROM marketing_ab
GROUP BY test_group, converted;
""", engine)

from statsmodels.stats.proportion import proportions_ztest
from scipy import stats

# z-test
success = df_ab[df_ab['converted']==1]['cnt'].values
nobs = df_ab.groupby("test_group")['cnt'].sum().values
z, p = proportions_ztest(success, nobs)
print(f"[전체 효과 - z-test] z = {z:.4f}, p-value = {p:.4e}")

# chi2 검정
table = df_ab.pivot(index="test_group", columns="converted", values="cnt")
chi2, p, dof, expected = stats.chi2_contingency(table)
print(f"[전체 효과 - chi2] Chi2 = {chi2:.4f}, p-value = {p:.4e}")

[전체 효과 - z-test] z = 7.3701, p-value = 1.7053e-13
[전체 효과 - chi2] Chi2 = 54.0058, p-value = 1.9990e-13


> z-test, chi2 모두 p < 0.05 → 귀무가설 기각  
> 광고 그룹과 PSA 그룹의 전환율 차이는 통계적으로 유의하다

#### 3-2. 요일 효과
- **H0 (귀무가설):** 요일과 그룹에 따른 전환율은 차이는 없다.
- **H1 (대립가설):** 요일과 그룹에 따른 전환율은 차이는 존재한다.
<hr>

In [72]:
df_day_ab = pd.read_sql("""
SELECT most_ads_day, test_group, converted, COUNT(*) AS cnt
FROM marketing_ab
GROUP BY most_ads_day, test_group, converted;
""", engine)

table_day = df_day_ab.pivot_table(index=["most_ads_day","test_group"], 
                                  columns="converted", values="cnt").fillna(0)
chi2, p, dof, expected = stats.chi2_contingency(table_day)
print(f"[요일 효과] Chi2 = {chi2:.4f}, p-value = {p:.4e}")

[요일 효과] Chi2 = 481.0886, p-value = 1.5146e-94


> p < 0.05 → 귀무가설 기각  
> 요일과 그룹에 따른 전환율 차이는 통계적으로 유의하다

#### 3-3. 시간대 효과
- **H0 (귀무가설):** 시간대와 그룹에 따른 전환율은 차이는 없다.
- **H1 (대립가설):** 시간대와 그룹에 따른 전환율은 차이는 존재한다.
<hr>

In [73]:
# 3. 시간대 효과
df_hour_ab = pd.read_sql("""
SELECT most_ads_hour, test_group, converted, COUNT(*) AS cnt
FROM marketing_ab
GROUP BY most_ads_hour, test_group, converted;
""", engine)

table_hour = df_hour_ab.pivot_table(index=["most_ads_hour","test_group"], 
                                    columns="converted", values="cnt").fillna(0)
chi2, p, dof, expected = stats.chi2_contingency(table_hour)
print(f"[시간대 효과] Chi2 = {chi2:.4f}, p-value = {p:.4e}")

[시간대 효과] Chi2 = 496.7429, p-value = 2.1638e-76


> p < 0.05 → 귀무가설 기각  
> 시간대와 그룹에 따른 전환율 차이는 통계적으로 유의하다

#### 3-4. 노출량 효과
- **H0 (귀무가설):** 광고 노출량은 전환율에 영향을 주지 않는다.
- **H1 (대립가설):** 광고 노출량은 전환율에 영향을 준다.
<hr>

In [69]:
# 4. 노출량 효과 (로지스틱 회귀)
import statsmodels.formula.api as smf

df_ads_ab = pd.read_sql("SELECT * FROM marketing_ab;", engine)
df_ads_ab['converted'] = df_ads_ab['converted'].astype(int)

model = smf.logit("converted ~ C(test_group) + total_ads", data=df_ads_ab).fit()
print(model.summary())

# odds ratio
import numpy as np
odds_ratios = np.exp(model.params)
print("\n승산비 (odds ratio):")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.108986
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:              converted   No. Observations:               588101
Model:                          Logit   Df Residuals:                   588098
Method:                           MLE   Df Model:                            2
Date:                Tue, 26 Aug 2025   Pseudo R-squ.:                 0.07467
Time:                        17:08:28   Log-Likelihood:                -64095.
converged:                       True   LL-Null:                       -69267.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -4.0412      0.010   -398.367      0.000      -4.061      -4.021

> 로지스틱 회귀 결과: 광고 그룹 변수 계수 유의, total_ads 양의 계수  
> → 광고 노출량이 많을수록 전환 확률 증가  
> → 광고 그룹이 PSA 그룹 대비 전환율 높음

### 4. Tableau
> 앞선 SQL + Python 분석에서는 데이터 전처리, 탐색, A/B Test 결과를 수치적으로 확인하였다.  
> 시각화 단계에서는 Tableau를 활용하여 **보다 직관적인 인사이트를 제공**한다.

- **대시보드 구성 요소**
  * 그룹별 전환율 비교
  * 요일·시간대별 전환율 분포
  * 광고 노출량에 따른 전환율 추이
  * 세그먼트별 교차 분석
 
- **파일 참조**
  * Tableau 대시보드 파일(`.twbx`) 별도 첨부